In [ ]:
nbit = 4
# llamasize = '7B'
llamasize = '13B'
temp = 0.1

In [ ]:
import pandas as pd
import glob
import pickle

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"

output_data_path = base_path + "/../Llama-2/data/output/2023.11.28-3_LLama2_13B-chat-hf_4-bit_float-16_5-few-shot_SEMANTIC_SIMILARITY"

drive.mount(mount_point, force_remount=True)
# data_path = '/content/gdrive/MyDrive/Colab/SDG/Data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx'

Mounted at /content/gdrive


In [ ]:
data = pd.read_excel(input_data_path)
data.head(3)

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN


In [ ]:
data_sel = data

In [ ]:
files = glob.glob(output_data_path + "/*.pkl")
# files

In [ ]:
results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

In [ ]:
len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['Id', 'text_id', 'sdg', 'result'])
results_processed.head(3)

,Id,text_id,sdg,result
0,20,1376,0,---SDG 0---
1,21,3889,0,CUDA out of memory. Tried to allocate 854.00 M...
2,22,6576,0,---SDG 3---


In [ ]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed.head(3)

,Id,text_id,sdg,result,label
0,20,1376,0,---SDG 0---,0
1,21,3889,0,CUDA out of memory. Tried to allocate 854.00 M...,NaN
2,22,6576,0,---SDG 3---,3


In [ ]:
final = data_sel.merge(results_processed.drop(columns=['sdg']), left_on='text_id', right_on='text_id')
final.head(3)

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,Id,result,label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,Sure! Here's the answer:\n\n---SDG 11---\n\nT...,11
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,---SDG 16---,16


In [ ]:
# final.groupby(['label']).count()

In [ ]:
# final.label.values

In [ ]:
final = final[~final.label.isna()]

In [ ]:
final.to_excel(output_data_path+"/2023.11.28-3_LLama2_13B-chat-hf_4-bit_float-16_5-few-shot_SEMANTIC_SIMILARITY.xlsx")

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

998
998
              precision    recall  f1-score   support

           0       0.88      0.34      0.49        44
           1       0.79      0.90      0.84        60
           2       0.93      0.87      0.90        60
           3       0.82      0.92      0.87        60
           4       0.93      0.93      0.93        60
           5       0.89      0.97      0.93        60
           6       1.00      0.97      0.98        60
           7       0.95      0.88      0.91        59
           8       0.69      0.71      0.70        59
           9       0.83      0.87      0.85        60
          10       0.76      0.69      0.73        59
          11       0.81      0.88      0.85        59
          12       0.86      0.98      0.91        60
          13       0.87      0.91      0.89        58
          14       0.98      0.95      0.97        60
          15       0.90      0.92      0.91        60
          16       0.93      0.90      0.92        60
          17       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classification_report_filename = output_data_path+f'/sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}_classification_report.xlsx'
classification_report_filename

'/content/gdrive/MyDrive/Colab/SDG/data/../Llama-2/data/output/2023.11.28-3_LLama2_13B-chat-hf_4-bit_float-16_5-few-shot_SEMANTIC_SIMILARITY/sdg_classification_llama_1020-texts_temp-0.1_nbit-4_llamasize-13B_classification_report.xlsx'

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test = pd.read_excel(classification_report_filename)
test

,Unnamed: 0,precision,recall,f1-score,support
0,0,0.882353,0.340909,0.491803,44.000000
1,1,0.794118,0.900000,0.843750,60.000000
2,2,0.928571,0.866667,0.896552,60.000000
3,3,0.820896,0.916667,0.866142,60.000000
4,4,0.933333,0.933333,0.933333,60.000000
5,5,0.892308,0.966667,0.928000,60.000000
6,6,1.000000,0.966667,0.983051,60.000000
7,7,0.945455,0.881356,0.912281,59.000000
8,8,0.688525,0.711864,0.700000,59.000000
9,9,0.825397,0.866667,0.845528,60.000000
